In [1]:
!nvidia-smi

Thu Mar 31 08:18:36 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:04:00.0 Off |                  N/A |
| 40%   49C    P8    31W / 260W |      1MiB / 11016MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:81:00.0 Off |                  N/A |
| 56%   

In [2]:
#@markdown ### Install `qqdm`
# Check if installed
try:
    import qqdm
except:
    ! pip install qqdm > /dev/null 2>&1
print("Done!")

Done!


In [3]:
workspace_dir = '.'


In [4]:
from PIL import Image
from IPython.display import display
for i in range(10, 20):
    im = Image.open("Omniglot/images_background/Japanese_(hiragana).0/character13/0500_" + str (i) + ".png")
    display(im)

In [5]:
# Import modules we need
import glob, random
from collections import OrderedDict

import numpy as np
import json

try:
    from qqdm.notebook import qqdm as tqdm
except ModuleNotFoundError:
    from tqdm.auto import tqdm

import torch, torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms

from PIL import Image
from IPython.display import display
import math
import os
import cv2
import matplotlib.pyplot as plt
import copy
import matplotlib.patches as patches
device = "cuda" if torch.cuda.is_available() else "cpu"

# fix random seeds
random_seed = 0
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(random_seed)

In [6]:
def ConvBlock(in_ch: int, out_ch: int):
    return nn.Sequential(
        nn.Conv2d(in_ch, out_ch, 3, padding=1),
        nn.BatchNorm2d(out_ch),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )

def ConvBlock4(in_ch: int, out_ch: int):
    return nn.Sequential(
        nn.Conv2d(in_ch, out_ch, 3, padding=1),
        nn.BatchNorm2d(out_ch),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=1)
    )


def ConvBlockFunction(x, w, b, w_bn, b_bn):
    x = F.conv2d(x, w, b, padding=1)
    x = F.batch_norm(x,
                     running_mean=None,
                     running_var=None,
                     weight=w_bn, bias=b_bn,
                     training=True)
    x = F.relu(x)
    x = F.max_pool2d(x, kernel_size=2, stride=2)
    return x


def ConvBlockFunction4(x, w, b, w_bn, b_bn):
    x = F.conv2d(x, w, b, padding=1)
    x = F.batch_norm(x,
                     running_mean=None,
                     running_var=None,
                     weight=w_bn, bias=b_bn,
                     training=True)
    x = F.relu(x)
    x = F.max_pool2d(x, kernel_size=2, stride=1)
    return x

In [7]:
def create_label(n_way, k_shot):
    return (torch.arange(n_way)
                 .repeat_interleave(k_shot)
                 .long())

# Try to create labels for 5-way 2-shot setting
create_label(5, 2)

tensor([0, 0, 1, 1, 2, 2, 3, 3, 4, 4])

In [8]:
def calculate_accuracy(logits, val_label):
    """ utility function for accuracy calculation """
    acc = np.asarray([(
        torch.argmax(logits, -1).cpu().numpy() == val_label.cpu().numpy())]
        ).mean() 
    return acc

In [9]:
def predict_true(logits, val_label):
    """ utility function for accuracy calculation """
    acc = np.asarray([(
        torch.argmax(logits, -1).cpu().numpy() == val_label.cpu().numpy())]
        )
    acc= acc[0]
    res=[]
    for x in acc:
        res =res+[bool(x)]
    return res

In [10]:
class Omniglot(Dataset):
    def __init__(self, data_dir, k_way, q_query):
        self.file_list = [f for f in glob.glob(
            data_dir + "**/character*", 
            recursive=True)]
        self.transform = transforms.Compose(
                            [transforms.ToTensor()])
        self.n = k_way + q_query

    def __getitem__(self, idx):
        sample = np.arange(20)

        # For random sampling the characters we want.
        np.random.shuffle(sample) 
        img_path = self.file_list[idx]
        img_list = [f for f in glob.glob(
            img_path + "**/*.png", recursive=True)]
        img_list.sort()
        imgs = [self.transform(
            Image.open(img_file).resize((28,28)).convert('RGB'))
#             Image.open(img_file).resize((56,56),Image.ANTIALIAS).convert('RGB'))
            for img_file in img_list]
        # `k_way + q_query` examples for each character
        imgs = torch.stack(imgs)[sample[:self.n]] 
        return imgs

    def __len__(self):
        return len(self.file_list)

In [11]:
def inner_update_MAML(fast_weights, loss, inner_lr):
    """ Inner Loop Update """
    grads = torch.autograd.grad(
        loss, fast_weights.values(), create_graph=True)
    # Perform SGD
    fast_weights = OrderedDict(
        (name, param - inner_lr * grad)
        for ((name, param), grad) in zip(fast_weights.items(), grads))
    return fast_weights

In [12]:
def collect_gradients_MAML(
    special_grad: OrderedDict, fast_weights, model, len_data):
    """ Actually do nothing (just backwards later) """
    return special_grad

In [13]:
def outer_update_MAML(model, meta_batch_loss, grad_tensors):
    """ Simply backwards """
    meta_batch_loss.backward()

In [14]:
n_way = 5
k_shot = 5
q_query = 15
inner_train_step = 1
inner_lr = 0.4
meta_lr = 0.001
meta_batch_size = 32
max_epoch = 30
eval_batches = test_batches = 20
train_data_path = './Omniglot/images_background/'
test_data_path = './Omniglot/images_evaluation/'    

In [15]:
def dataloader_init(datasets, num_workers=4):
    train_set, val_set, test_set = datasets
    train_loader = DataLoader(train_set,
                            # The "batch_size" here is not \
                            #    the meta batch size, but  \
                            #    how many different        \
                            #    characters in a task,     \
                            #    i.e. the "n_way" in       \
                            #    few-shot classification.
                            batch_size=n_way,
                            num_workers=num_workers,
                            shuffle=True,
                            drop_last=True)
    val_loader = DataLoader(val_set,
                            batch_size=n_way,
                            num_workers=num_workers,
                            shuffle=True,
                            drop_last=True)
    test_loader = DataLoader(test_set,
                            batch_size=n_way,
                            num_workers=num_workers,
                            shuffle=True,
                            drop_last=True)
    train_iter = iter(train_loader)
    val_iter = iter(val_loader)
    test_iter = iter(test_loader)
    return (train_loader, val_loader, test_loader), \
           (train_iter, val_iter, test_iter)

train_set, val_set = torch.utils.data.random_split(
    Omniglot(train_data_path, k_shot, q_query), [3200, 656])
test_set = Omniglot(test_data_path, k_shot, q_query)

(train_loader, val_loader, test_loader), \
(train_iter, val_iter, test_iter) = dataloader_init(
                             (train_set, val_set, test_set))

In [16]:
def get_meta_batch(meta_batch_size,
                   k_shot, q_query, 
                   data_loader, iterator):
    data = []
    for _ in range(meta_batch_size):
        try:
            # a "task_data" tensor is representing \
            #     the data of a task, with size of \
            #     [n_way, k_shot+q_query, 1, 28, 28]
            task_data = iterator.next()  
        except StopIteration:
            iterator = iter(data_loader)
            task_data = iterator.next()
        train_data = (task_data[:, :k_shot]
                      .reshape(-1, 3, 28, 28))
        val_data = (task_data[:, k_shot:]
                    .reshape(-1, 3,28, 28))
        task_data = torch.cat(
            (train_data, val_data), 0)
        data.append(task_data)
    return torch.stack(data).to(device), iterator

In [17]:
# meta_model=torch.load('best')

In [18]:
class newClassifier(nn.Module):
    def __init__(self, in_ch, k_way):
        super(newClassifier, self).__init__()
        self.conv1 = ConvBlock(in_ch, 64)
        self.conv2 = ConvBlock(64, 64)
        self.conv3 = ConvBlock(64, 64)
        self.conv4 = ConvBlock4(64, 64)
#         self.add_on_layers = nn.Sequential(
#             nn.Conv2d(64,128,kernel_size=1),
#             nn.ReLU(),
#             nn.Conv2d(128,128,kernel_size=1),
#             nn.Sigmoid()
#         )
        prototype_shape = (20,64,1,1)
        self.prototype_shape = prototype_shape
        self.img_size = 28
        self.num_prototypes = prototype_shape[0]
        self.num_classes = k_way
        self.epsilon = 1e-4
        self.prototype_activation_function =  'log'
        assert(self.num_prototypes % self.num_classes == 0)
        self.prototype_class_identity = torch.zeros(self.num_prototypes,
                                                    self.num_classes)
        num_prototypes_per_class = self.num_prototypes // self.num_classes
        for j in range(self.num_prototypes):
            self.prototype_class_identity[j, j // num_prototypes_per_class] = 1
            
        self.prototype_vectors = nn.Parameter(torch.rand(self.prototype_shape),
                                              requires_grad=True)
        self.ones = nn.Parameter(torch.ones(self.prototype_shape),
                                 requires_grad=False)
        self.logits = nn.Linear(self.num_prototypes, k_way
                                    ,bias=False
                               ) # do not use bias    
        self._initialize_weights()
    
    
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = x.view(x.shape[0], -1)
        x = self.logits(x)
        return x
    
    
    def set_last_layer_incorrect_connection(self, incorrect_strength):
        '''
        the incorrect strength will be actual strength if -0.5 then input -0.5
        '''
        positive_one_weights_locations = torch.t(self.prototype_class_identity)
        negative_one_weights_locations = 1 - positive_one_weights_locations

        correct_class_connection = 1
        incorrect_class_connection = incorrect_strength
        self.logits.weight.data.copy_(
            correct_class_connection * positive_one_weights_locations
            + incorrect_class_connection * negative_one_weights_locations)
    
    
    def functional_forward(self, x, params):
        '''
        Arguments:
        x: input images [batch, 1, 28, 28]
        params: model parameters, 
                i.e. weights and biases of convolution
                     and weights and biases of 
                                   batch normalization
                type is an OrderedDict

        Arguments:
        x: input images [batch, 1, 28, 28]
        params: The model parameters, 
                i.e. weights and biases of convolution 
                     and batch normalization layers
                It's an `OrderedDict`
        '''
        for block in [1, 2,3]:
            x = ConvBlockFunction(
                x,
                params[f'conv{block}.0.weight'],
                params[f'conv{block}.0.bias'],
                params.get(f'conv{block}.1.weight'),
                params.get(f'conv{block}.1.bias'))
        for block in [4]:
            x = ConvBlockFunction4(
                x,
                params[f'conv{block}.0.weight'],
                params[f'conv{block}.0.bias'],
                params.get(f'conv{block}.1.weight'),
                params.get(f'conv{block}.1.bias'))

        x2 = x **2
        x2_patch_sum = F.conv2d(x,params['ones'])

        p2 = params['prototype_vectors'] ** 2
        p2 = torch.sum(p2, dim=(1, 2, 3))
        p2_reshape = p2.view(-1, 1, 1)
        xp = F.conv2d(x,params['prototype_vectors'])
        intermediate_result = - 2 * xp + p2_reshape  # use broadcast
        # x2_patch_sum and intermediate_result are of the same shape
        distances = F.relu(x2_patch_sum + intermediate_result)
        
        min_distances = -F.max_pool2d(-distances,
                              kernel_size=(distances.size()[2],
                                           distances.size()[3]))
        min_distances = min_distances.view(-1, self.num_prototypes)
        
        prototype_activations = torch.log((min_distances + 1) / (min_distances + self.epsilon))
        logits = F.linear(prototype_activations,params['logits.weight'])
#         x = x.view(x.shape[0], -1)
#         x = F.linear(x,
#                      params['logits.weight'])
#                      params['logits.bias'])
        return logits,min_distances
     
    
    def _initialize_weights(self):
#         for m in self.add_on_layers.modules():
#             if isinstance(m, nn.Conv2d):
#                 # every init technique has an underscore _ in the name
#                 nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')

#                 if m.bias is not None:
#                     nn.init.constant_(m.bias, 0)

#             elif isinstance(m, nn.BatchNorm2d):
#                 nn.init.constant_(m.weight, 1)
#                 nn.init.constant_(m.bias, 0)
        self.set_last_layer_incorrect_connection(incorrect_strength=-0.5)
    
    
    def push_forward(self, x,params):
        '''this method is needed for the pushing operation'''
        for block in [1, 2, 3]:
            x = ConvBlockFunction(
                x,
                params[f'conv{block}.0.weight'],
                params[f'conv{block}.0.bias'],
                params.get(f'conv{block}.1.weight'),
                params.get(f'conv{block}.1.bias'))
        for block in [4]:
            x = ConvBlockFunction4(
                x,
                params[f'conv{block}.0.weight'],
                params[f'conv{block}.0.bias'],
                params.get(f'conv{block}.1.weight'),
                params.get(f'conv{block}.1.bias'))
        conv_output=x
        
        x2 = x ** 2
        x2_patch_sum = F.conv2d(input=x2, weight=params['ones'])
        p2 = params['prototype_vectors'] ** 2
        p2 = torch.sum(p2, dim=(1, 2, 3))
        # p2 is a vector of shape (num_prototypes,)
        # then we reshape it to (num_prototypes, 1, 1)
        p2_reshape = p2.view(-1, 1, 1)

        xp = F.conv2d(input=x, weight=params['prototype_vectors'])
        intermediate_result = - 2 * xp + p2_reshape  # use broadcast
        # x2_patch_sum and intermediate_result are of the same shape
        distances = F.relu(x2_patch_sum + intermediate_result)
        return conv_output, distances

    


In [19]:

def model_init_new():
    meta_multi = newClassifier(3, n_way).to(device)
#     optimizer = torch.optim.Adam(meta_model.parameters(), 
#                                  lr=meta_lr)
    loss_fn = nn.CrossEntropyLoss().to(device)
    return meta_multi, loss_fn


meta_multi, loss_fn = model_init_new()
joint_optimizer_lrs = {'features': 1e-4,# 有改动，原来1e-4
                       'add_on_layers': 3e-3,
                       'prototype_vectors':1e-3}
# no weight_decay!!!
joint_optimizer_specs = \
[
    {'params': meta_multi.conv1.parameters(), 'lr': joint_optimizer_lrs['features']}, # bias are now also being regularized
 {'params': meta_multi.conv2.parameters(), 'lr': joint_optimizer_lrs['features']},
 {'params': meta_multi.conv3.parameters(), 'lr': joint_optimizer_lrs['features']},
 {'params': meta_multi.conv4.parameters(), 'lr': joint_optimizer_lrs['features']},
#  {'params': meta_multi.add_on_layers.parameters(), 'lr': joint_optimizer_lrs['add_on_layers']},
 {'params': meta_multi.prototype_vectors, 'lr': joint_optimizer_lrs['prototype_vectors']},
# {'params': meta_multi.logits.parameters(), 'lr':1e-4}
]
joint_optimizer = torch.optim.Adam(joint_optimizer_specs)

In [20]:
# meta_multi=torch.load('pp_10epoch_3max1stride_64ch_12_15_-2sep')

In [21]:
#helper.py
def makedir(path):
    '''
    if path does not exist in the file system, create it
    '''
    if not os.path.exists(path):
        os.makedirs(path)



def find_high_activation_crop(activation_map, percentile=60):
    threshold = np.percentile(activation_map, percentile)
    mask = np.ones(activation_map.shape)
    mask[activation_map < threshold] = 0
    lower_y, upper_y, lower_x, upper_x = 0, 0, 0, 0
    for i in range(mask.shape[0]):
        if np.amax(mask[i]) > 0.5:
            lower_y = i
            break
    for i in reversed(range(mask.shape[0])):
        if np.amax(mask[i]) > 0.5:
            upper_y = i
            break
    for j in range(mask.shape[1]):
        if np.amax(mask[:,j]) > 0.5:
            lower_x = j
            break
    for j in reversed(range(mask.shape[1])):
        if np.amax(mask[:,j]) > 0.5:
            upper_x = j
            break
    return lower_y, upper_y+1, lower_x, upper_x+1

def coumpute_zjym_prototype(size_ori,size_now,position):
    patch=size_ori // size_now
    fmap_height_start_index = position[1] * patch
    fmap_height_end_index = (position[1]+1)*patch
    fmap_width_start_index = position[2] * patch
    fmap_width_end_index = (position[2]+1)*patch    
    return [position[0],fmap_height_start_index,fmap_height_end_index,fmap_width_start_index,fmap_width_end_index]

In [22]:
def SimilarityConsis(search_batch_input,
                       prototype_network_parallel,
                       fast_weights,
                       search_y=None, # required if class_specific == True
                       num_classes=None):

    search_batch = search_batch_input
    with torch.no_grad():
        search_batch = search_batch.cuda()
        # this computation currently is not parallelized
        _, proto_dist_torch = meta_multi.push_forward(search_batch,fast_weights)

    proto_dist_ = np.copy(proto_dist_torch.detach().cpu().numpy())

    del proto_dist_torch

    class_to_img_index_dict = {key: [] for key in range(n_way)}
        # img_y is the image's integer label
    for img_index, img_y in enumerate(search_y):
        img_label = img_y.item()
        class_to_img_index_dict[img_label].append(img_index)

    prototype_shape = meta_multi.prototype_shape
    n_prototypes = prototype_shape[0]
    x= list(np.zeros((q_query,n_prototypes)))
    for j in range(n_prototypes):
        target_class = torch.argmax(meta_multi.prototype_class_identity[j]).item()
        proto_dist_j = proto_dist_[class_to_img_index_dict[target_class]][:,j,:,:]# class_to_img_index_dict[target_class]为原型所属的类，[:,j,:,:]为第j个原型。意思是这个类下的图片和第j个原型的结果。
        for m in range(len(proto_dist_j)):
            quer_proto_dist_j = proto_dist_j[m]
            index=np.argmin(quer_proto_dist_j, axis=None)
            x[m][j]=int(index)
    res = 0
    for i in range(len(x)):
        if res ==0:
            res=[[]]
        else:
            res.append([[]])
        for j in range(len(x[0])):
            res[i]=res[i]+[int(x[i][j])]
    return res

In [23]:
# def plusBB(img,x0,x1,y0,y1,piccurrent):
#     fig = plt.figure(28)
#     ax1 = plt.subplot(111)
#     p = plt.imshow(img,vmin=0,vmax=1,shape=(28,28))
#     upleft = (x0, y0)
#     width = x1-x0
#     height = y1-y0
#     rect = patches.Rectangle(upleft, width, height, linewidth=1, edgecolor='r', facecolor='none')
#     ax1.add_patch(rect) 
#     ax1.axis('off')
#     fig = plt.gcf()
#     fig.savefig('sss2.png', dpi=1, transparent=True, pad_inches=0, bbox_inches='tight')
# #     del fig,ax1

In [24]:
# x, train_iter = get_meta_batch(
#             meta_batch_size, k_shot, q_query, 
#             train_loader, train_iter)
# for task_index,meta_batch in enumerate(x):
# #             print(task_index)
#     support_set = meta_batch[: n_way * k_shot]  
#     query_set = meta_batch[n_way * k_shot :]    

#     fast_weights = OrderedDict(meta_multi.named_parameters())
#     break
# original_img_j=support_set[0]
# original_img_j = original_img_j.data.cpu().numpy()
# original_img_j = np.transpose(original_img_j, (1, 2, 0))
# original_img_size = original_img_j.shape[0]
# plusBB(original_img_j,5,10,10,20,'sss.png')

In [25]:
def update_prototypes_on_batch(search_batch_input,
                               start_index_of_search_batch,
                               prototype_network_parallel,
                               global_min_proto_dist, # this will be updated
                               global_min_fmap_patches, # this will be updated
                               proto_rf_boxes, # this will be updated
                               proto_bound_boxes, # this will be updated
                               fast_weights,
                               class_specific=True,
                               search_y=None, # required if class_specific == True
                               num_classes=None, # required if class_specific == True
                               prototype_layer_stride=1,
                               dir_for_saving_prototypes=None,
                               prototype_img_filename_prefix=None,
                               prototype_self_act_filename_prefix=None,
                               prototype_activation_function_in_numpy=None):

#     prototype_network_parallel.eval()


        # print('preprocessing input for pushing ...')
        # search_batch = copy.deepcopy(search_batch_input)
    # 可能有问题
#     search_batch = preprocess(search_batch_input)
    search_batch = search_batch_input
    with torch.no_grad():
        search_batch = search_batch.cuda()
        # this computation currently is not parallelized
        protoL_input_torch, proto_dist_torch = prototype_network_parallel.push_forward(search_batch,fast_weights)

    protoL_input_ = np.copy(protoL_input_torch.detach().cpu().numpy())
    proto_dist_ = np.copy(proto_dist_torch.detach().cpu().numpy())

    del protoL_input_torch, proto_dist_torch

    if class_specific:
        class_to_img_index_dict = {key: [] for key in range(num_classes)}
        # img_y is the image's integer label
        for img_index, img_y in enumerate(search_y):
            img_label = img_y.item()
            class_to_img_index_dict[img_label].append(img_index)

    prototype_shape = prototype_network_parallel.prototype_shape
    n_prototypes = prototype_shape[0]
    proto_h = prototype_shape[2]
    proto_w = prototype_shape[3]
    max_dist = prototype_shape[1] * prototype_shape[2] * prototype_shape[3]

    for j in range(n_prototypes):
        #if n_prototypes_per_class != None:
        if class_specific:
            # target_class is the class of the class_specific prototype
            target_class = torch.argmax(prototype_network_parallel.prototype_class_identity[j]).item()
            # if there is not images of the target_class from this batch
            # we go on to the next prototype
            if len(class_to_img_index_dict[target_class]) == 0:
                continue
            proto_dist_j = proto_dist_[class_to_img_index_dict[target_class]][:,j,:,:]# class_to_img_index_dict[target_class]为原型所属的类，[:,j,:,:]为第j个原型。意思是这个类下的图片和第j个原型的结果。
        else:
            # if it is not class specific, then we will search through
            # every example
            proto_dist_j = proto_dist_[:,j,:,:]

        batch_min_proto_dist_j = np.amin(proto_dist_j)# 第j个原型所属的第x类的最近的值。（激活最大的地方）
        if batch_min_proto_dist_j < global_min_proto_dist[j]:# 
            batch_argmin_proto_dist_j = \
                list(np.unravel_index(np.argmin(proto_dist_j, axis=None),
                                      proto_dist_j.shape))# batch最小原型距离的坐标。
            if class_specific:
                '''
                change the argmin index from the index among
                images of the target class to the index in the entire search
                batch
                '''
                # 因为都是五分类，这一步可以省略。
                batch_argmin_proto_dist_j[0] = class_to_img_index_dict[target_class][batch_argmin_proto_dist_j[0]]

            # retrieve the corresponding feature map patch
            img_index_in_batch = batch_argmin_proto_dist_j[0]
            fmap_height_start_index = batch_argmin_proto_dist_j[1] * prototype_layer_stride
            fmap_height_end_index = fmap_height_start_index + proto_h
            fmap_width_start_index = batch_argmin_proto_dist_j[2] * prototype_layer_stride
            fmap_width_end_index = fmap_width_start_index + proto_w
            # 对应的features(x),对应的位置。
            batch_min_fmap_patch_j = protoL_input_[img_index_in_batch,
                                                   :,
                                                   fmap_height_start_index:fmap_height_end_index,
                                                   fmap_width_start_index:fmap_width_end_index]
            # 此处应该会修改，毕竟每个五分类任务的训练集只有一个。
            global_min_proto_dist[j] = batch_min_proto_dist_j
            global_min_fmap_patches[j] = batch_min_fmap_patch_j
            
            # get the receptive field boundary of the image patch
            # that generates the representation
#             protoL_rf_info = prototype_network_parallel.proto_layer_rf_info
#             rf_prototype_j = compute_rf_prototype(search_batch.size(2), batch_argmin_proto_dist_j, protoL_rf_info)
            rf_prototype_j = coumpute_zjym_prototype(search_batch.size(2),protoL_input_.shape[2],batch_argmin_proto_dist_j)
            # get the whole image
            original_img_j = search_batch_input[rf_prototype_j[0]]
            original_img_j = original_img_j.data.cpu().numpy()
            original_img_j = np.transpose(original_img_j, (1, 2, 0))
            original_img_size = original_img_j.shape[0]#28,28,1
            
            # crop out the receptive field
            rf_img_j = original_img_j[rf_prototype_j[1]:rf_prototype_j[2],
                                      rf_prototype_j[3]:rf_prototype_j[4], :]
            
            # save the prototype receptive field information
            proto_rf_boxes[j, 0] = rf_prototype_j[0] + start_index_of_search_batch
            proto_rf_boxes[j, 1] = rf_prototype_j[1]
            proto_rf_boxes[j, 2] = rf_prototype_j[2]
            proto_rf_boxes[j, 3] = rf_prototype_j[3]
            proto_rf_boxes[j, 4] = rf_prototype_j[4]
            if proto_rf_boxes.shape[1] == 6 and search_y is not None:# 记录类别。
                proto_rf_boxes[j, 5] = search_y[rf_prototype_j[0]].item()

            # find the highly activated region of the original image
            proto_dist_img_j = proto_dist_[img_index_in_batch, j, :, :]
            if prototype_network_parallel.prototype_activation_function == 'log':
                proto_act_img_j = np.log((proto_dist_img_j + 1) / (proto_dist_img_j + prototype_network_parallel.epsilon))
            elif prototype_network_parallel.prototype_activation_function == 'linear':
                proto_act_img_j = max_dist - proto_dist_img_j
            else:
                proto_act_img_j = prototype_activation_function_in_numpy(proto_dist_img_j)
            upsampled_act_img_j = cv2.resize(proto_act_img_j, dsize=(original_img_size, original_img_size),
                                             interpolation=cv2.INTER_CUBIC)
            proto_bound_j = find_high_activation_crop(upsampled_act_img_j)
            # crop out the image patch with high activation as prototype image
            proto_img_j = original_img_j[proto_bound_j[0]:proto_bound_j[1],
                                         proto_bound_j[2]:proto_bound_j[3], :]

            # save the prototype boundary (rectangular boundary of highly activated region)
            proto_bound_boxes[j, 0] = proto_rf_boxes[j, 0]
            proto_bound_boxes[j, 1] = proto_bound_j[0]
            proto_bound_boxes[j, 2] = proto_bound_j[1]
            proto_bound_boxes[j, 3] = proto_bound_j[2]
            proto_bound_boxes[j, 4] = proto_bound_j[3]
            if proto_bound_boxes.shape[1] == 6 and search_y is not None:
                proto_bound_boxes[j, 5] = search_y[rf_prototype_j[0]].item()

            if dir_for_saving_prototypes is not None:
                if prototype_self_act_filename_prefix is not None:
                    # save the numpy array of the prototype self activation
                    np.save(os.path.join(dir_for_saving_prototypes,
                                         prototype_self_act_filename_prefix + str(j) + '.npy'),
                            proto_act_img_j)
                if prototype_img_filename_prefix is not None:
                    # save the whole image containing the prototype as png
#                     plusBB(original_img_j,proto_bound_j[0],proto_bound_j[1],proto_bound_j[2],proto_bound_j[3],os.path.join(dir_for_saving_prototypes,
#                                             prototype_img_filename_prefix + '-original' + str(j) + '.png'))
#                     cv2.rectangle(original_img_j,(int(proto_bound_j[0]),int(proto_bound_j[1])),(int(proto_bound_j[2]),int(proto_bound_j[3])),(0,255,0),3)  
                    plt.imsave(os.path.join(dir_for_saving_prototypes,
                                            prototype_img_filename_prefix + '-original' + str(j) + '.png'),
                               original_img_j,
#                                np.squeeze(original_img_j),
                               vmin=0.0,
                               vmax=1.0)
                    # overlay (upsampled) self activation on original image and save the result
                    rescaled_act_img_j = upsampled_act_img_j - np.amin(upsampled_act_img_j)
                    rescaled_act_img_j = rescaled_act_img_j / np.amax(rescaled_act_img_j)
                    heatmap = cv2.applyColorMap(np.uint8(255*rescaled_act_img_j), cv2.COLORMAP_JET)
                    heatmap = np.float32(heatmap) / 255
                    heatmap = heatmap[...,::-1]
                    overlayed_original_img_j = 0.5 * original_img_j + 0.3 * heatmap
                    plt.imsave(os.path.join(dir_for_saving_prototypes,
                                            prototype_img_filename_prefix + '-original_with_self_act' + str(j) + '.png'),
                               overlayed_original_img_j,
                               vmin=0.0,
                               vmax=1.0)
                    
                    # if different from the original (whole) image, save the prototype receptive field as png
                    if rf_img_j.shape[0] != original_img_size or rf_img_j.shape[1] != original_img_size:
                        plt.imsave(os.path.join(dir_for_saving_prototypes,
                                                prototype_img_filename_prefix + '-receptive_field' + str(j) + '.png'),
#                                    np.squeeze(rf_img_j),
                                   rf_img_j,
                                   vmin=0.0,
                                   vmax=1.0)
                        overlayed_rf_img_j = overlayed_original_img_j[rf_prototype_j[1]:rf_prototype_j[2],
                                                                      rf_prototype_j[3]:rf_prototype_j[4]]
                        plt.imsave(os.path.join(dir_for_saving_prototypes,
                                                prototype_img_filename_prefix + '-receptive_field_with_self_act' + str(j) + '.png'),
                                   overlayed_rf_img_j,
                                   vmin=0.0,
                                   vmax=1.0)
                    
                    # save the prototype image (highly activated region of the whole image)
                    plt.imsave(os.path.join(dir_for_saving_prototypes,
                                            prototype_img_filename_prefix + str(j) + '.png'),
#                                 np.squeeze(proto_img_j),
                                proto_img_j,
                               vmin=0.0,
                               vmax=1.0)
                
    if class_specific:
        del class_to_img_index_dict


In [26]:


def toList(tensor):
    res = np.copy(tensor.detach().cpu().numpy())
    return res.tolist()


In [27]:
meta_multi = torch.load('best2')

In [28]:
def MetaAlgorithmGeneratorNew(
    inner_update      = inner_update_MAML, 
    collect_gradients = collect_gradients_MAML, 
    outer_update      = outer_update_MAML):

    global calculate_accuracy

    def MetaAlgorithmNew(
        model, optimizer, x, n_way, k_shot, q_query, loss_fn,epoch_number,step,
        inner_train_step=8, inner_lr=0.04, train=0): 
        criterion = loss_fn
        task_loss, task_acc = [], []
        special_grad = OrderedDict()  # Added for variants!
        coefs = {
            'crs_ent': 1,
            'clst': 0.8,
            'sep': -0.8,
            'l1': 1e-4,
            }
        task_crsEnt,task_cls,task_sep,task_l1 = [],[],[],[]
        task_dic={}
        task_dic_path='./test'
        makedir(task_dic_path)
        for task_index,meta_batch in enumerate(x):
#             print(task_index)
            support_set = meta_batch[: n_way * k_shot]  
            query_set = meta_batch[n_way * k_shot :]    
            
            fast_weights = OrderedDict(model.named_parameters())
            
            ### ---------- INNER TRAIN LOOP ---------- ###
            for inner_step in range(inner_train_step): 
                train_label = create_label(n_way, k_shot).to(device)
                logits,min_distances = model.functional_forward(support_set, fast_weights)
                cross_entropy = criterion(logits, train_label)

                max_dist = (model.prototype_shape[1]
                            * model.prototype_shape[2]
                            * model.prototype_shape[3])
                # calculate cluster cost                
                prototypes_of_correct_class = torch.t(model.prototype_class_identity[:,train_label]).cuda()
                inverted_distances, _ = torch.max((max_dist - min_distances) * prototypes_of_correct_class, dim=1)
                cluster_cost = torch.mean(max_dist - inverted_distances)
                
                
                # calculate separation cost
                prototypes_of_wrong_class = 1 - prototypes_of_correct_class
                inverted_distances_to_nontarget_prototypes, _ = \
                    torch.max((max_dist - min_distances) * prototypes_of_wrong_class, dim=1)
                separation_cost = torch.mean(max_dist - inverted_distances_to_nontarget_prototypes)
                    
                # calculate avg cluster cost
                avg_separation_cost = \
                    torch.sum(min_distances * prototypes_of_wrong_class, dim=1) / torch.sum(prototypes_of_wrong_class, dim=1)
                avg_separation_cost = torch.mean(avg_separation_cost)
                
                
                l1_mask = 1 - torch.t(model.prototype_class_identity).cuda()
                l1 = (model.logits.weight * l1_mask).norm(p=1)
                loss = (coefs['crs_ent'] * cross_entropy
                                  + coefs['clst'] * cluster_cost
                                  + coefs['sep'] * separation_cost
                                  + coefs['l1'] * l1)
                ones =  fast_weights.pop('ones')
                logitstemp = fast_weights.pop('logits.weight')
                fast_weights = inner_update(fast_weights, loss, inner_lr)
                
                fast_weights['ones'] = ones
                fast_weights['logits.weight'] = logitstemp
            
            # push prototype
            if epoch_number>14:
                train_label = create_label(n_way,k_shot).to(device)

                prototype_shape = model.prototype_shape
                n_prototypes = model.num_prototypes
                global_min_proto_dist = np.full(n_prototypes, np.inf)
                global_min_fmap_patches = np.zeros(
                    [n_prototypes,
                     prototype_shape[1],
                     prototype_shape[2],
                     prototype_shape[3]])
                proto_rf_boxes = np.full(shape=[n_prototypes, 6],
                                    fill_value=-1)
                proto_bound_boxes = np.full(shape=[n_prototypes, 6],
                                            fill_value=-1)
                makedir('./img')
                root_dir_for_saving_prototypes = os.path.join('./img', 'epoch-'+str(epoch_number))
                task_dic_path = os.path.join(task_dic_path,'epoch-'+str(epoch_number))
                if train==0:
                    root_dir_for_saving_prototypes = os.path.join(root_dir_for_saving_prototypes, 'train')
                elif train==1:
                    root_dir_for_saving_prototypes = os.path.join(root_dir_for_saving_prototypes, 'val')
                else:
                    root_dir_for_saving_prototypes = os.path.join('./test','epoch-'+str(epoch_number))
                    root_dir_for_saving_prototypes =os.path.join(root_dir_for_saving_prototypes,'prototype')
                root_dir_for_saving_prototypes = os.path.join(root_dir_for_saving_prototypes,'step-'+str(step))
                
                makedir(root_dir_for_saving_prototypes)
                # 此处的epoch_number可能要传epoch进来
#                 print(os.path.join(root_dir_for_saving_prototypes,'task-'+str(task_index)))
                proto_epoch_dir =os.path.join(root_dir_for_saving_prototypes,'task-'+str(task_index))
                
                makedir(proto_epoch_dir)
                search_batch_size =n_way

                num_classes = model.num_classes
    #             for push_iter, search_batch_inpu in eupdate_prototypes_on_batchnumerate(support_set):

                start_index_of_search_batch = 0

                update_prototypes_on_batch(support_set,
                                   start_index_of_search_batch,
                                   model,
                                   global_min_proto_dist,
                                   global_min_fmap_patches,
                                   proto_rf_boxes,
                                   proto_bound_boxes,
                                   fast_weights,
                                   class_specific=True,
                                   search_y=train_label,
                                   num_classes=num_classes,
                                   prototype_layer_stride=1,
                                   dir_for_saving_prototypes=proto_epoch_dir,
                                   prototype_img_filename_prefix='prototype-img',
                                   prototype_self_act_filename_prefix='prototype-self-act')    

                proto_bound_boxes_filename_prefix='bb'
                # proto_bound_boxes_filename_prefix ='bb'=='None'
                if proto_epoch_dir != None and proto_bound_boxes_filename_prefix != None:
                    np.save(os.path.join(proto_epoch_dir, proto_bound_boxes_filename_prefix + str(epoch_number) + '.npy'),
                            proto_bound_boxes)     

                prototype_update = np.reshape(global_min_fmap_patches,
                                      tuple(prototype_shape))
                
                fast_weights['prototype_vectors'] = torch.tensor(prototype_update, dtype=torch.float32).cuda()
            
            
            
            ### ---------- INNER VALID LOOP ---------- ###
            # ----inner test-----#
            val_label = create_label(n_way, q_query).to(device)
            # FIXME: W for val?
            ones =  fast_weights.pop('ones')            
            logitstemp = fast_weights.pop('logits.weight')
            
            special_grad = collect_gradients(
                special_grad, fast_weights, model, len(x))
            
            fast_weights['ones'] = ones
            fast_weights['logits.weight'] = logitstemp
            # Collect gradients for outer loop
            logits,min_distances = model.functional_forward(query_set, fast_weights) 
            
            cross_entropy = criterion(logits, val_label)
            all_dic={}
            index_dic={}
            high_dic={}
            if train ==2:
                if epoch_number>14:
                    task_dic_path = os.path.join('./test','epoch-'+str(epoch_number))
                    task_dic_path = os.path.join(task_dic_path,'original')
                    task_dic_path = os.path.join(task_dic_path,'step-'+str(step))
                    task_dic_path = os.path.join(task_dic_path,'task-'+str(task_index))
                    makedir(task_dic_path)
    #                 pp_distance=min_distances.cpu().numpy()
                    for t in range(len(val_label)):
                        original_img_j = query_set[t]
                        original_img_j = original_img_j.data.cpu().numpy()
                        original_img_j = np.transpose(original_img_j, (1, 2, 0))
                        original_img_size = original_img_j.shape[0]
                        if t % q_query ==0:
                            final_path = os.path.join(task_dic_path,'class-'+str(t//q_query))
                            makedir(final_path)
                            
                        plt.imsave(os.path.join(final_path,str(t%q_query)+'.png'),
                               original_img_j,
                               vmin=0.0,
                               vmax=1.0)
                        
                    task_dic['task-'+str(task_index)]=toList(min_distances)
                    all_dic['res']=predict_true(logits,val_label)
                    high_index= SimilarityConsis(query_set,model,fast_weights,search_y=val_label,num_classes=num_classes)
                    index_dic['index']=high_index
                    high_score =  torch.log((min_distances + 1) / (min_distances + model.epsilon))
                    high_dic['res']=toList(high_score)
                    jsondata = json.dumps(task_dic,indent=4)
                    jsondata2 = json.dumps(all_dic,indent=4)
                    jsondata3 = json.dumps(index_dic,indent=4)
                    jsondata4 = json.dumps(high_dic,indent=4)
                    f = open(task_dic_path+'/distance.json', 'w')
                    f2= open(task_dic_path+'/predict_res.json','w')
                    f3 =open(task_dic_path+'/index.json','w')
                    f4=open(task_dic_path+'/activation.json','w')
                    f.write(jsondata)
                    f2.write(jsondata2)
                    f3.write(jsondata3)
                    f4.write(jsondata4)
                    f.close()
                    f2.close()
                    f3.close()
                    f4.close()
            max_dist = (model.prototype_shape[1]
                * model.prototype_shape[2]
                * model.prototype_shape[3])
            
            # calculate cluster cost                
            prototypes_of_correct_class = torch.t(model.prototype_class_identity[:,val_label]).cuda()
            inverted_distances, _ = torch.max((max_dist - min_distances) * prototypes_of_correct_class, dim=1)
            cluster_cost = torch.mean(max_dist - inverted_distances)


            # calculate separation cost
            prototypes_of_wrong_class = 1 - prototypes_of_correct_class
            inverted_distances_to_nontarget_prototypes, _ = \
                torch.max((max_dist - min_distances) * prototypes_of_wrong_class, dim=1)
            separation_cost = torch.mean(max_dist - inverted_distances_to_nontarget_prototypes)

            # calculate avg cluster cost
            avg_separation_cost = \
                torch.sum(min_distances * prototypes_of_wrong_class, dim=1) / torch.sum(prototypes_of_wrong_class, dim=1)
            avg_separation_cost = torch.mean(avg_separation_cost)
            l1_mask = 1 - torch.t(model.prototype_class_identity).cuda()
            l1 = (model.logits.weight * l1_mask).norm(p=1)
            
            loss = (coefs['crs_ent'] * cross_entropy
                              + coefs['clst'] * cluster_cost
                              + coefs['sep'] * separation_cost
                              + coefs['l1'] * l1)
            
            task_loss.append(loss)
            task_acc.append(calculate_accuracy(logits, val_label))
            task_crsEnt.append(cross_entropy)
            task_cls.append(cluster_cost)
            task_sep.append(separation_cost)
            task_l1.append(l1)
#             task_acc.append(calculate_accuracy(logits, val_label))
        # Update outer loop
        model.train()
        optimizer.zero_grad()

        meta_batch_loss = torch.stack(task_loss).mean()
        meta_batch_crsEnt=torch.stack(task_crsEnt).mean()
        meta_batch_cls =torch.stack(task_cls).mean()
        meta_batch_sep = torch.stack(task_sep).mean()
        meta_batch_l1 = torch.stack(task_l1).mean()
        if train==0:
            ones =  fast_weights.pop('ones')
            logitstemp = fast_weights.pop('logits.weight')
            # Notice the update part!
            outer_update(model, meta_batch_loss, special_grad)
            fast_weights['ones'] = ones
            fast_weights['logits.weight'] = logitstemp            
            optimizer.step()
        task_acc = np.mean(task_acc)
        return meta_batch_loss, task_acc,meta_batch_crsEnt,meta_batch_cls,meta_batch_sep,meta_batch_l1
    return MetaAlgorithmNew

In [29]:
MAML_new = MetaAlgorithmGeneratorNew()
MetaAlgorithm_new =MAML_new

In [30]:
# torch.load("ssss")

In [31]:
# max_epoch=20
# for epoch in range(max_epoch):
#     print("Epoch %d" % (epoch + 1))
#     train_meta_loss = []
#     train_acc = []
#     train_meta_crsEnt,train_meta_cls,train_meta_sep,train_meta_l1=[],[],[],[]
#     # The "step" here is a meta-gradinet update step
#     for step in tqdm(range(
#             len(train_loader) // meta_batch_size)): 
#         x, train_iter = get_meta_batch(
#             meta_batch_size, k_shot, q_query, 
#             train_loader, train_iter)
# #         meta_loss, acc = MetaAlgorithm_new(
#         meta_loss, acc,meta_crsEnt,meta_cls,meta_sep,meta_l1 = MetaAlgorithm_new(
#             meta_multi, joint_optimizer, x, 
#             n_way, k_shot, q_query, loss_fn,epoch,step)
#         train_meta_loss.append(meta_loss.item())
#         train_acc.append(acc)
#         train_meta_crsEnt.append(meta_crsEnt.item())
#         train_meta_cls.append(meta_cls.item())
#         train_meta_sep.append(meta_sep.item())
#         train_meta_l1.append(meta_l1.item())

#     print("  Loss    : ", "%.3f" % (np.mean(train_meta_loss)), end='\t')
#     print("  Accuracy: ", "%.3f %%" % (np.mean(train_acc) * 100),end='\t')
#     print("  crsEnt    : ", "%.3f" % (np.mean(train_meta_crsEnt)), end='\t')
#     print("  cls    : ", "%.3f" % (np.mean(train_meta_cls)), end='\t')
#     print("  sep    : ", "%.3f" % (np.mean(train_meta_sep)), end='\t')
#     print("  l1    : ", "%.3f" % (np.mean(train_meta_l1)))

# #     See the validation accuracy after each epoch.
# #     Early stopping is welcomed to implement.
#     val_acc = []
#     for eval_step in tqdm(range(
#             len(val_loader) // (eval_batches))):
#         x, val_iter = get_meta_batch(
#             eval_batches, k_shot, q_query, 
#             val_loader, val_iter)
#         # We update three inner steps when testing.
# #         _, acc = MetaAlgorithm_new(meta_multi, joint_optimizer, x, 
#         _, acc,_,_,_,_ = MetaAlgorithm_new(meta_multi, joint_optimizer, x, 
#             n_way, k_shot, q_query, 
#                       loss_fn,
#                       epoch,
#                       eval_step,
#                       inner_train_step=12, 
#                       train=False) 
#         val_acc.append(acc)
#     print("  Validation accuracy: ", "%.3f %%" % (np.mean(val_acc) * 100))
#     if np.mean(val_acc)>0.98:
#         break

In [32]:

test_acc = []
for test_step in tqdm(range(
        len(test_loader) // (test_batches))):
    x, test_iter = get_meta_batch(
        test_batches, k_shot, q_query, 
        test_loader, test_iter)
    # When testing, we update 3 inner-steps
    _, acc,_,_,_,_ = MetaAlgorithm_new(meta_multi, joint_optimizer, x, 
            n_way, k_shot, q_query, 
                      loss_fn,
                      27,
                      test_step,
                      inner_train_step=10, 
                      train=2) 
    test_acc.append(acc)
print("  Testing accuracy: ", "%.3f %%" % (np.mean(test_acc) * 100))

 Iters    Elapsed Time      Speed                                               
  6/6   00:00:29<00:00:00  0.20it/s                                             

IpythonBar(children=(HTML(value='  0.0%'), FloatProgress(value=0.0)))

  Testing accuracy:  95.656 %


In [33]:
# torch.save(meta_multi,'完全体')

In [34]:
# torch.save(meta_multi,'完全体_3max1stride_noadd_64ch_10_12_20prototype')

In [35]:
# torch.save(meta_multi,'完全体3max1stride_noadd_64ch_10_12')

In [36]:
# meta_multi= torch.load('完全体3max1stride_noadd_64ch_10_12')